In [16]:
from pathlib import Path

import numpy as np
from scipy.linalg import expm
import plotly.express as px

In [2]:
q_mat_path = Path("..", "raw_data", "vehicle_ages", "q_matrix.csv")
q_mat = np.loadtxt(fname=q_mat_path, delimiter=",", ndmin=2)

In [3]:
s_mat = q_mat[1:,1:]
s_mat_sup_0 = q_mat[1: , 0]

In [4]:
alpha = np.zeros(shape=(1, 10))
alpha[0, 0] = 1

In [28]:
def age_prior_pdf(x, alpha, s_mat, s_mat_sup_0):
    """Compute the probability density function
    
    of the age prior at x years.
    See: https://en.m.wikipedia.org/wiki/Phase-type_distribution
    """
    return (alpha @ expm(s_mat * x) @ s_mat_sup_0).item(0)

def age_prior_survival(x, alpha, s_mat, s_mat_sup_0):
    """Compute the survival function
    
    of the age prior at x years.
    See: https://en.m.wikipedia.org/wiki/Phase-type_distribution
    """
    return (alpha @ expm(s_mat * x) @ np.ones(shape=(alpha.shape[1], 1))).item(0)

In [40]:
age_prior_pdf_vals = [age_prior_pdf(x, alpha=alpha, s_mat=s_mat, s_mat_sup_0=s_mat_sup_0) for x in range(1, 256, 1)]

In [41]:
px.line(x = range(1, 256, 1), y = age_prior_pdf_vals)

In [31]:
age_prior_survival_vals = [age_prior_survival(x, alpha=alpha, s_mat=s_mat, s_mat_sup_0=s_mat_sup_0) for x in range(1, 256, 1)]

In [33]:
px.line(x = range(1, 256, 1), y = age_prior_survival_vals)

In [ ]:
# The probability of seeing a car of a given age can be modeled 
# as the ratio of the probability of a car surviving that long
# to it surviving to an age of 2025 - 1769 = 256.
# Imagine a situation where cars are 99% likely to survive until
# their first birthday, 98% likely to survive until their 2nd birthday,
# and 97% likely to survive until their 3rd birthday (and cars cannot get to their 3rd birthday).
# Then, the probability of seeing a 0-year-old car is 99/(99+98+97) = 0.3367,
# a 1-year-old car is 0.3333, and a 2-year-old car is 0.3299.

In [36]:
age_prior_pdf_vals_2 = [age_prior_survival_vals[i] / sum(age_prior_survival_vals) for i in range(len(age_prior_survival_vals))]

In [38]:
px.line(x=[range(len(age_prior_pdf_vals_2))], y=age_prior_pdf_vals_2)